In [5]:
# Particle Swarm Optimization (PSO) za UPFC parametre

import numpy as np
from scipy.integrate import solve_ivp
import random

class PowerSystemModel:
    """Model elektroenergetskog sistema iz XML-a"""
    def __init__(self):
        # Osnovni parametri sistema
        self.wn = 376.99111843077515
        self.ws = 1
        self.pi = 3.141592653589793

        # Parametri generatora (iz XML-a)
        self.M1, self.M2, self.M3 = 35, 10.0, 4.5
        self.D1_base, self.D2_base, self.D3_base = 12.0, 8.0, 6.0
        self.T1d1, self.T1d2, self.T1d3 = 8.96, 6.0, 5.89

        # Reaktanse generatora
        self.xd1, self.xq1, self.x1d1 = 0.146, 0.0969, 0.0608
        self.xd2, self.xq2, self.x1d2 = 0.8958, 0.8645, 0.1198
        self.xd3, self.xq3, self.x1d3 = 1.3125, 1.2578, 0.1813

        # UPFC parametri (iz XML-a)
        self.X_se_upfc = 0.085
        self.b_se_upfc = 11.7647
        self.P_ref_upfc = 0.1
        self.V_ref_upfc = 1.0
        self.T_upfc = 0.5

        # Ograničenja UPFC-a
        self.r_upfc_max = 0.02
        self.r_upfc_min = 0.0001
        self.gamma_max = 5
        self.gamma_min = -5

        # Nominalni naponi i uglovi (iz XML-a)
        self.V1_0, self.V2_0, self.V3_0 = 1.04, 1.025, 1.025
        self.V4_0, self.V5_0, self.V6_0 = 1.0, 1.0, 1.0
        self.V7_0, self.V8_0, self.V9_0 = 1.0, 1.0, 1.0

        # Opterećenja (iz XML-a)
        self.pl5_0, self.ql5_0 = 0.9, 0.3
        self.pl7_0, self.ql7_0 = 1.0, 0.35
        self.pl9_0, self.ql9_0 = 1.25, 0.5

        # Generacija (iz XML-a)
        self.pg1_0, self.qg1_0 = 0.723, 0.2703
        self.pg2_0, self.qg2_0 = 1.63, 0.0654
        self.pg3_0, self.qg3_0 = 0.85, -0.10949999999999999

        # Admitanse mreže - normalne vrednosti za liniju 6-7
        self.G6_7_normal = -1.155087480890097
        self.B6_7_normal = 9.784270426363173
        self.G7_6_normal = -1.155087480890097
        self.B7_6_normal = 9.784270426363173

        # Ostale admitanse (samo one koje utiču na stabilnost)
        self.G4_4, self.B4_4 = 3.3073789620253065, -39.30888872611897
        self.G9_9, self.B9_9 = 2.5527920926017282, -17.338230096448523
        self.G4_9, self.B4_9 = -1.36518771331058, 11.60409556313993
        self.G9_4, self.B9_4 = -1.36518771331058, 11.60409556313993

        # Parametri kvara
        self.fault_start = 1.0
        self.fault_end = 1.15

    def system_dynamics(self, t, x, params):
        """Dinamika sistema sa UPFC kontrolom i kvarom"""
        # Parametri za optimizaciju
        Kp_upfc, Ki_upfc, Kd_upfc, D1_extra, D2_extra, D3_extra = params

        # State varijable
        delta1, w1, e1q1, id1, iq1 = x[0:5]
        delta2, w2, e1q2, id2, iq2 = x[5:10]
        delta3, w3, e1q3, id3, iq3 = x[10:15]
        upfc_error_p, upfc_error_v = x[15:17]
        r_upfc, gamma_upfc = x[17:19]

        # Simulacija kvara na liniji 6-7 (tačno kao u XML-u)
        fault_active = self.fault_start < t <= self.fault_end

        if fault_active:
            # Tokom kvara - linija 6-7 je isključena
            G6_7 = 0
            B6_7 = 0
            G7_6 = 0
            B7_6 = 0
            # Ovo utiče na distribuciju snage kroz mrežu
            fault_impact_factor = 1.5  # Pojačana nestabilnost
        else:
            # Normalna operacija
            G6_7 = self.G6_7_normal
            B6_7 = self.B6_7_normal
            G7_6 = self.G7_6_normal
            B7_6 = self.B7_6_normal
            fault_impact_factor = 1.0

        # Aproksimacija napona na bazi steady-state (pojednostavljeno)
        V1 = self.V1_0 + 0.01 * np.sin(delta1)  # Mala varijacija oko nom. vrednosti
        V2 = self.V2_0 + 0.01 * np.sin(delta2)
        V3 = self.V3_0 + 0.01 * np.sin(delta3)
        V4 = self.V4_0 + 0.02 * np.sin(delta1 - delta3)  # UPFC utiče ovde
        V9 = self.V9_0 + 0.02 * np.sin(delta3)

        # Theta uglovi (pojednostavljeno)
        theta1 = delta1 * 0.1  # Aproksimacija
        theta2 = delta2 * 0.1
        theta3 = delta3 * 0.1
        theta4 = (delta1 + delta3) * 0.05
        theta9 = delta3 * 0.1

        # UPFC kontrolni signali (kao u XML-u)
        P_actual = V4 * V9 * self.b_se_upfc * np.sin(theta4 - theta9)
        upfc_error_p_new = (self.P_ref_upfc - P_actual) / self.T_upfc
        upfc_error_v_new = (self.V_ref_upfc - V9) / self.T_upfc

        # UPFC kontrola (PID)
        r_upfc_dot = (1/self.T_upfc) * (
            Kp_upfc * (self.P_ref_upfc - P_actual) +
            Ki_upfc * upfc_error_p +
            Kd_upfc * (upfc_error_p_new - upfc_error_p)
        )

        gamma_upfc_dot = (1/self.T_upfc) * (
            Kp_upfc * (self.V_ref_upfc - V9) +
            Ki_upfc * upfc_error_v +
            Kd_upfc * (upfc_error_v_new - upfc_error_v)
        )

        # Ograničavanje UPFC parametara
        r_upfc_bounded = np.clip(r_upfc, self.r_upfc_min, self.r_upfc_max)
        gamma_upfc_bounded = np.clip(gamma_upfc, self.gamma_min, self.gamma_max)

        # UPFC uticaj na snagu (pojednostavljeno, na bazi XML strukture)
        upfc_power_injection = 0.1 * r_upfc_bounded * self.b_se_upfc * V4 * V9 * np.sin(theta4 - theta9 + gamma_upfc_bounded * self.pi/180)

        # Električne snage generatora (uključujući uticaj kvara i UPFC-a)
        # Generator 1 - povezan na bus 4, direktno utiče UPFC
        Pe1 = (e1q1 * iq1 + (self.xq1 - self.x1d1) * id1 * iq1) + upfc_power_injection * 0.3

        # Generator 2 - udaljen od UPFC-a, manje uticaja
        Pe2 = (e1q2 * iq2 + (self.xq2 - self.x1d2) * id2 * iq2) * fault_impact_factor

        # Generator 3 - povezan na bus 6, najviše pogođen kvarom linije 6-7
        Pe3 = (e1q3 * iq3 + (self.xq3 - self.x1d3) * id3 * iq3) * fault_impact_factor * 1.2 - upfc_power_injection * 0.7

        # Swing jednačine generatora (sa poboljšanim damping-om)
        delta1_dot = self.wn * (w1 - self.ws)
        w1_dot = (1/self.M1) * (self.pg1_0 - Pe1 - (self.D1_base + D1_extra) * (w1 - self.ws))

        delta2_dot = self.wn * (w2 - self.ws)
        w2_dot = (1/self.M2) * (self.pg2_0 - Pe2 - (self.D2_base + D2_extra) * (w2 - self.ws))

        delta3_dot = self.wn * (w3 - self.ws)
        w3_dot = (1/self.M3) * (self.pg3_0 - Pe3 - (self.D3_base + D3_extra) * (w3 - self.ws))

        # Generator field flux equations (pojednostavljeno)
        # Pretpostavljamo konstantne field napone za pojednostavljenje
        vf1 = 1.0  # Konstantan field napon
        vf2 = 1.0
        vf3 = 1.0

        e1q1_dot = (1/self.T1d1) * (-e1q1 - (self.xd1 - self.x1d1) * id1 + vf1)
        e1q2_dot = (1/self.T1d2) * (-e1q2 - (self.xd2 - self.x1d2) * id2 + vf2)
        e1q3_dot = (1/self.T1d3) * (-e1q3 - (self.xd3 - self.x1d3) * id3 + vf3)

        # Algebraic constraints za id i iq (pojednostavljeno)
        # Ovo bi trebalo da bude algebraic, ali za ODE aproksimiramo
        id1_target = -V1 * np.sin(theta1 - delta1) / self.xq1
        iq1_target = (e1q1 - V1 * np.cos(theta1 - delta1)) / self.x1d1

        id2_target = -V2 * np.sin(theta2 - delta2) / self.xq2
        iq2_target = (e1q2 - V2 * np.cos(theta2 - delta2)) / self.x1d2

        id3_target = -V3 * np.sin(theta3 - delta3) / self.xq3
        iq3_target = (e1q3 - V3 * np.cos(theta3 - delta3)) / self.x1d3

        # Brza dinamika za id i iq (vremenske konstante su male)
        tau_fast = 0.01
        id1_dot = (id1_target - id1) / tau_fast
        iq1_dot = (iq1_target - iq1) / tau_fast
        id2_dot = (id2_target - id2) / tau_fast
        iq2_dot = (iq2_target - iq2) / tau_fast
        id3_dot = (id3_target - id3) / tau_fast
        iq3_dot = (iq3_target - iq3) / tau_fast

        return [
            delta1_dot, w1_dot, e1q1_dot, id1_dot, iq1_dot,
            delta2_dot, w2_dot, e1q2_dot, id2_dot, iq2_dot,
            delta3_dot, w3_dot, e1q3_dot, id3_dot, iq3_dot,
            upfc_error_p_new, upfc_error_v_new,
            r_upfc_dot, gamma_upfc_dot
        ]

class PSO_Optimizer:
    def __init__(self, power_system):
        self.power_system = power_system
        self.n_particles = 30
        self.n_iterations = 50
        self.w = 0.7  # Inertia weight
        self.c1 = 1.5  # Cognitive parameter
        self.c2 = 1.5  # Social parameter

        # Granice parametara [Kp_upfc, Ki_upfc, Kd_upfc, D1_extra, D2_extra, D3_extra]
        # Na bazi trenutnih vrednosti iz XML-a
        self.bounds_min = np.array([0.001, 0.001, 0.0001, 50, 20, 10])
        self.bounds_max = np.array([0.1, 0.2, 0.05, 800, 400, 300])
        self.n_params = len(self.bounds_min)

    def objective_function(self, params):
        """Ciljna funkcija - minimizacija nestabilnosti sistema"""
        try:
            # Početni uslovi (na bazi XML InitProc)
            delta1_0, delta2_0, delta3_0 = 0.1, 0.1, 0.1
            w1_0, w2_0, w3_0 = 1.0, 1.0, 1.0
            e1q1_0, e1q2_0, e1q3_0 = 1.0, 1.0, 1.0
            id1_0, iq1_0 = 1.0, 1.0
            id2_0, iq2_0 = 1.0, 1.0
            id3_0, iq3_0 = 1.0, 1.0
            upfc_error_p_0, upfc_error_v_0 = 0.0, 0.0
            r_upfc_0, gamma_upfc_0 = 0.001, 0.0

            x0 = [
                delta1_0, w1_0, e1q1_0, id1_0, iq1_0,
                delta2_0, w2_0, e1q2_0, id2_0, iq2_0,
                delta3_0, w3_0, e1q3_0, id3_0, iq3_0,
                upfc_error_p_0, upfc_error_v_0,
                r_upfc_0, gamma_upfc_0
            ]

            # Vremenski opseg (fokus na prvh 5 sekundi)
            t_span = (0, 5.0)
            t_eval = np.linspace(0, 5.0, 1000)

            # Rešavanje diferencijalnih jednačina
            sol = solve_ivp(
                lambda t, x: self.power_system.system_dynamics(t, x, params),
                t_span, x0, t_eval=t_eval, method='RK45', rtol=1e-6, atol=1e-8
            )

            if not sol.success:
                return 1e6  # Velika penalizacija za neuspešnu simulaciju

            # Izdvajanje promenljivih stanja
            delta1, w1 = sol.y[0], sol.y[1]
            delta2, w2 = sol.y[2], sol.y[6]
            delta3, w3 = sol.y[4], sol.y[11]

            # Kriterijumi stabilnosti

            # 1. Maksimalno odstupanje rotor uglova (kritično za stabilnost)
            max_delta_dev = max(
                np.max(np.abs(delta1 - delta1_0)),
                np.max(np.abs(delta2 - delta2_0)),
                np.max(np.abs(delta3 - delta3_0))
            )

            # 2. Maksimalno odstupanje frekvencije
            max_freq_dev = max(
                np.max(np.abs(w1 - 1.0)),
                np.max(np.abs(w2 - 1.0)),
                np.max(np.abs(w3 - 1.0))
            )

            # 3. Relativna stabilnost između generatora
            delta12 = delta1 - delta2
            delta13 = delta1 - delta3
            delta23 = delta2 - delta3

            max_relative_angle = max(
                np.max(np.abs(delta12)),
                np.max(np.abs(delta13)),
                np.max(np.abs(delta23))
            )

            # 4. Brzina promene rotor uglova (indikator oscilacija)
            delta_rates = np.array([
                np.max(np.abs(np.diff(delta1))),
                np.max(np.abs(np.diff(delta2))),
                np.max(np.abs(np.diff(delta3)))
            ])
            max_delta_rate = np.max(delta_rates)

            # 5. Post-fault stabilnost (poslednjih 2 sekunde)
            if len(t_eval) > 400:
                post_fault_period = sol.y[:15, -400:]  # Poslednje 2 sekunde
                post_fault_std = np.std(post_fault_period, axis=1)
                post_fault_instability = np.sum(post_fault_std)
            else:
                post_fault_instability = 0

            # 6. UPFC efikasnost (da li uspešno kontroliše sistem)
            r_upfc_values = sol.y[17] if len(sol.y) > 17 else np.zeros_like(t_eval)
            upfc_efficiency = np.mean(np.abs(r_upfc_values - 0.01))  # Željeno oko 0.01

            # 7. Penalizacija za ekstremne vrednosti
            extreme_penalty = 0
            if max_delta_dev > 2.0:  # Rotor uglovi ne smeju biti preveliki
                extreme_penalty += (max_delta_dev - 2.0) * 1000
            if max_freq_dev > 0.1:  # Frekvencija mora biti stabilna
                extreme_penalty += (max_freq_dev - 0.1) * 2000

            # Kombinovana ciljna funkcija (sve u istim jedinicama)
            objective = (
                max_delta_dev * 50 +           # Stabilnost rotor uglova
                max_freq_dev * 200 +           # Stabilnost frekvencije
                max_relative_angle * 30 +      # Relativna stabilnost
                max_delta_rate * 100 +         # Brzina oscilacija
                post_fault_instability * 10 +  # Post-fault stabilnost
                upfc_efficiency * 20 +         # UPFC efikasnost
                extreme_penalty                # Penalizacija ekstrema
            )

            return objective

        except Exception as e:
            print(f"Greška u simulaciji: {e}")
            return 1e6  # Velika penalizacija za greške

    def optimize(self):
        """PSO algoritam optimizacije"""
        print("Pokretanje PSO optimizacije...")
        print(f"Broj čestica: {self.n_particles}, Broj iteracija: {self.n_iterations}")

        # Inicijalizacija čestica
        particles = np.random.uniform(
            self.bounds_min, self.bounds_max,
            (self.n_particles, self.n_params)
        )

        # Inicijalizacija brzina
        velocities = np.random.uniform(-0.1, 0.1, (self.n_particles, self.n_params))

        # Evaluacija početne fitness
        fitness = np.array([self.objective_function(p) for p in particles])

        # Inicijalizacija personalnih i globalnih najboljih
        personal_best = particles.copy()
        personal_best_fitness = fitness.copy()
        global_best_idx = np.argmin(fitness)
        global_best = particles[global_best_idx].copy()
        global_best_fitness = fitness[global_best_idx]

        print(f"Početna najbolja fitness: {global_best_fitness:.4f}")

        # PSO iteracije
        for iteration in range(self.n_iterations):
            for i in range(self.n_particles):
                # Ažuriranje brzine
                r1, r2 = np.random.random(2)
                velocities[i] = (
                    self.w * velocities[i] +
                    self.c1 * r1 * (personal_best[i] - particles[i]) +
                    self.c2 * r2 * (global_best - particles[i])
                )

                # Ažuriranje pozicije
                particles[i] += velocities[i]

                # Primena granica
                particles[i] = np.clip(particles[i], self.bounds_min, self.bounds_max)

                # Evaluacija fitness
                current_fitness = self.objective_function(particles[i])

                # Ažuriranje personalnog najboljeg
                if current_fitness < personal_best_fitness[i]:
                    personal_best[i] = particles[i].copy()
                    personal_best_fitness[i] = current_fitness

                # Ažuriranje globalnog najboljeg
                if current_fitness < global_best_fitness:
                    global_best = particles[i].copy()
                    global_best_fitness = current_fitness

            # Ispis napretka svakih 10 iteracija
            if (iteration + 1) % 10 == 0:
                print(f"Iteracija {iteration + 1}: Najbolja fitness = {global_best_fitness:.4f}")

        return global_best, global_best_fitness

def main():
    """Glavna rutina optimizacije"""
    print("=== OPTIMIZACIJA UPFC PID PARAMETARA ===")
    print("Inicijalizacija modela elektroenergetskog sistema...")

    # Kreiranje modela sistema
    power_system = PowerSystemModel()

    # Kreiranje PSO optimizatora
    optimizer = PSO_Optimizer(power_system)

    # Pokretanje optimizacije
    best_params, best_fitness = optimizer.optimize()

    print("\n=== REZULTATI OPTIMIZACIJE ===")
    print(f"Najbolja fitness vrednost: {best_fitness:.6f}")
    print("\nOptimalni parametri:")
    print(f"Kp_upfc = {best_params[0]:.6f}")
    print(f"Ki_upfc = {best_params[1]:.6f}")
    print(f"Kd_upfc = {best_params[2]:.6f}")
    print(f"D1_extra = {best_params[3]:.2f}")
    print(f"D2_extra = {best_params[4]:.2f}")
    print(f"D3_extra = {best_params[5]:.2f}")

    print("\n=== XML FORMAT ZA VAŠ MODEL ===")
    print(f'<Param name="Kp_upfc" val="{best_params[0]:.6f}"/>')
    print(f'<Param name="Ki_upfc" val="{best_params[1]:.6f}"/>')
    print(f'<Param name="Kd_upfc" val="{best_params[2]:.6f}"/>')
    print(f'<Param name="D1_extra" val="{best_params[3]:.2f}"/>')
    print(f'<Param name="D2_extra" val="{best_params[4]:.2f}"/>')
    print(f'<Param name="D3_extra" val="{best_params[5]:.2f}"/>')

    # Poređenje sa trenutnim parametrima iz XML-a
    current_params = [0.01, 0.0976050, 0.0100, 500.0, 200.0, 150.0]
    current_fitness = optimizer.objective_function(current_params)

    print(f"\n=== POREĐENJE ===")
    print(f"Trenutni parametri fitness: {current_fitness:.6f}")
    print(f"Optimizovani parametri fitness: {best_fitness:.6f}")
    if current_fitness > 0:
        improvement = ((current_fitness - best_fitness)/current_fitness*100)
        print(f"Poboljšanje: {improvement:.2f}%")

if __name__ == "__main__":
    main()

=== OPTIMIZACIJA UPFC PID PARAMETARA ===
Inicijalizacija modela elektroenergetskog sistema...
Pokretanje PSO optimizacije...
Broj čestica: 30, Broj iteracija: 50
Početna najbolja fitness: 166.3033
Iteracija 10: Najbolja fitness = 164.2892
Iteracija 20: Najbolja fitness = 164.2884
Iteracija 30: Najbolja fitness = 164.2883
Iteracija 40: Najbolja fitness = 164.2883
Iteracija 50: Najbolja fitness = 164.2883

=== REZULTATI OPTIMIZACIJE ===
Najbolja fitness vrednost: 164.288265

Optimalni parametri:
Kp_upfc = 0.003723
Ki_upfc = 0.001000
Kd_upfc = 0.000100
D1_extra = 232.94
D2_extra = 400.00
D3_extra = 300.00

=== XML FORMAT ZA VAŠ MODEL ===
<Param name="Kp_upfc" val="0.003723"/>
<Param name="Ki_upfc" val="0.001000"/>
<Param name="Kd_upfc" val="0.000100"/>
<Param name="D1_extra" val="232.94"/>
<Param name="D2_extra" val="400.00"/>
<Param name="D3_extra" val="300.00"/>

=== POREĐENJE ===
Trenutni parametri fitness: 171.341500
Optimizovani parametri fitness: 164.288265
Poboljšanje: 4.12%
